In [2]:
#Copy paste API key and Password from paper trading account
password= "7zLQqlPDDYbuFJiB1OBUhHkRg9rHCy49hnKljeQz"
API = "PKR72SJF21YRGY2FJQ7C"

In [1]:
!pip install alpaca-py
!pip install alpaca-trade-api

In [5]:
from alpaca.trading.client import TradingClient
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.stream import TradingStream
from alpaca.trading.requests import MarketOrderRequest

client = TradingClient(API, password,paper = True)

In [7]:
import requests
import asyncio
import alpaca_trade_api as tradeapi

In [ ]:
#account = dict(client.get_account())
#account
#for key, value in account.items():
#    print(f'{key} : {value}')


In [ ]:
#This is how you place an order in Alpaca

In [47]:
order_details = MarketOrderRequest(
    symbol = "PLTR",
    qty = 10,
    side = OrderSide.BUY, #OrderSide is the class and BUY is the obj
    time_in_force = TimeInForce.DAY
)

order = client.submit_order(order_data = order_details) #synchronous
trades = TradingStream(API, password, paper = True) # async function > waits for confirmation from event loop

async def trade_status(data):
    print(data)

trades.subscribe_trade_updates(trade_status)

#Runs this only after an event loop
async def main():
    await trades._run_forever()

#check for an active event loop
try: 
    #if event loop is running
    loop = asyncio.get_running_loop()
    task = loop.create_task(main()) #Sched main coroutine
except RunTimeError:
    #No event loop, so we start one
    asyncio.run(main())
except Exception as e:
    print("error:", e)

print(client.get_order_by_id(order.id))


id=UUID('15318799-fda8-43ee-be50-950677c507f0') client_order_id='6a33b981-60d8-49e0-ad9d-c82b2fb41f99' created_at=datetime.datetime(2024, 9, 25, 20, 29, 13, 29584, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2024, 9, 25, 20, 29, 13, 30356, tzinfo=TzInfo(UTC)) submitted_at=datetime.datetime(2024, 9, 25, 20, 29, 13, 29584, tzinfo=TzInfo(UTC)) filled_at=None expired_at=None canceled_at=None failed_at=None replaced_at=None replaced_by=None replaces=None asset_id=UUID('80e04a93-e1a8-4503-ab61-2ef1992650b4') symbol='PLTR' asset_class=<AssetClass.US_EQUITY: 'us_equity'> notional=None qty='10' filled_qty='0' filled_avg_price=None order_class=<OrderClass.SIMPLE: 'simple'> order_type=<OrderType.MARKET: 'market'> type=<OrderType.MARKET: 'market'> side=<OrderSide.BUY: 'buy'> time_in_force=<TimeInForce.DAY: 'day'> limit_price=None stop_price=None status=<OrderStatus.ACCEPTED: 'accepted'> extended_hours=False legs=None trail_percent=None trail_price=None hwm=None


In [51]:
#define a function for long and short

def long(symbol: str, qty : int): 
    '''
    Very basic. Can add tim_in_force variants, limit_orders, etc
    '''
    try: 
        order_details = MarketOrderRequest(
            symbol = symbol,
            qty = qty,
            side = OrderSide.BUY,
            time_in_force = TimeInForce.DAY #Order validity.Customizable -> GTC: Good till cancel [GTD: till day], IOC: Immediate or Cancel, FOK: FIll or Kill, OPG: Opening or kill,CLS: Exec at market close
        )
        order = client.submit_order(order_data = order_details)

        #Wait for order filling and fetch details
        order_status = client.get_order_by_id(order.id)
        
        if order_status.status == 'filled' :
            filled_price = float(order_status.filled_avg_price)
            print(f' Long {symbol}@ {filled_price} x {qty}')
            return order.id
        else:
            print(f'Order status: {order_status.status}')
            return None
        
    except Exception as e:
        print(f'Failed to BUY order: {e}')
        return None
        

def short(symbol: str, qty : int): 
    '''
    Very basic. Can add tim_in_force variants, limit_orders, etc
    '''
    try: 
        order_details = MarketOrderRequest(
            symbol = symbol,
            qty = qty,
            side = OrderSide.SELL,
            time_in_force = TimeInForce.DAY #Order validity.Customizable -> GTC: Good till cancel [GTD: till day], IOC: Immediate or Cancel, FOK: FIll or Kill, OPG: Opening or kill,CLS: Exec at market close
        )
        order = client.submit_order(order_data = order_details)

        #Wait for order filling and fetch details
        order_status = client.get_order_by_id(order.id)
        
        if order_status.status =='filled':
            filled_price = float(order_status.filled_avg_price)
            print(f' Long {symbol}@ {filled_price} x {qty}')
            return order.id
        else:
            print(f'Order status: {order_status.status}')
            print(f'Order = {order_status}')
            return None
    except Exception as e:
        print(f'Failed to BUY order: {e}')
        return None
                    

In [53]:
#Order History
def order_book(order_id):
    order_status=client.get_order(order_id)
    if order_status.status =='filled':
        filled_price = float(order_status.filled_avg_price)
        return filled_price
    else:
        print(f'Order not filled : {order_status.status}')
        return None


In [109]:
def long(symbol: str, qty : int): 
    '''
    Very basic. Can add tim_in_force variants, limit_orders, etc
    '''
    try: 
        order_details = MarketOrderRequest(
            symbol = symbol,
            qty = qty,
            side = OrderSide.BUY,
            time_in_force = TimeInForce.DAY #Order validity.Customizable -> GTC: Good till cancel [GTD: till day], IOC: Immediate or Cancel, FOK: FIll or Kill, OPG: Opening or kill,CLS: Exec at market close
        )
        order = client.submit_order(order_data = order_details)

        #Wait for order filling and fetch details
        order_status = client.get_order_by_id(order.id)
        
        if order_status.status == 'filled' :
            filled_price = float(order_status.filled_avg_price)
            print(f' Long {symbol}@ {filled_price} x {qty}')
            return order.id
        else:
            print(f'Order status: {order_status.status}')
            return None
        time.sleep(2) # wait before checking whether the orer is filled or not
    except Exception as e:
        print(f'Failed to BUY order: {e}')
        return None

def track_order_status(order_id):
    try:
        while True:
            order_status = client.get_order_by_id(order_id)
            # Check if the order is filled
            if order_status.status == 'filled':
                filled_price = float(order_status.filled_avg_price)
                print(f'Order {order_id} filled at price: {filled_price}')
                return filled_price
            else:
                print(f'Order {order_id} status: {order_status.status}, waiting for it to fill...')
            
            # Wait before checking again
            time.sleep(2)
    
    except Exception as e:
        print(f'Error while tracking order: {e}')
        return None




Order status: OrderStatus.ACCEPTED
Order ID = None


In [75]:
# from alpaca.trading.models import OrderStatus
# from alpaca.trading.requests import GetOrdersRequest
# from alpaca.trading.enums import QueryOrderStatus

In [107]:
# # Get the last 100 closed orders
# get_orders_data = GetOrdersRequest(
#     status=QueryOrderStatus.CLOSED,
#     limit=100,
#     nested=True  # show nested multi-leg orders
# )
# client.get_orders(filter=get_orders_data)[1]

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('80e04a93-e1a8-4503-ab61-2ef1992650b4'),
    'canceled_at': None,
    'client_order_id': 'b899110f-d2d7-422c-9bc9-17ac8776d0aa',
    'created_at': datetime.datetime(2024, 9, 25, 18, 16, 34, 970007, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': datetime.datetime(2024, 9, 25, 18, 16, 35, 832521, tzinfo=TzInfo(UTC)),
    'filled_avg_price': '36.9',
    'filled_qty': '1',
    'hwm': None,
    'id': UUID('e083a8b9-a0ac-4c71-8eb8-fdbc250ccc3d'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.FILLED: 'filled'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 9, 25, 18,

In [ ]:
#Now we have to implement MACD strategy



